# Produce grid of monthly Pb fields

In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import pickle
import matplotlib
import xarray as xr
import glob
import matplotlib.backends.backend_pdf

%matplotlib inline

#### Parameters:

In [2]:
# domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179
jsize=jmax-jmin
isize=imax-imin

# Resolution of grid cells to show in figure:
grid_reso  = 10

land_color  = '#a9a7a2'

#### Load files:

In [3]:
mfile        = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = mfile['nav_lon'].values
mesh_lat     = mfile['nav_lat'].values
mask         = mfile['tmask'].values[0]
mesh_bathy   = mfile['tmask'].values[0,:,:,:]
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
mesh_depth   = mfile['nav_lev'].values

In [4]:
files = glob.glob(f'/data/brogalla/run_storage/Pb-tuning-202302/Pb-Hudson-Bay-2002_20230217/ANHA12_EXH006_5d_*')

In [5]:
years  = np.arange(2002,2009,1)
files1 = []; files2 = [];
for year in years:
    folder1 = f'/data/brogalla/run_storage/Pb-tuning-202302/Pb-Hudson-Bay-{year}_20230217/'
    folder2 = f'/data/brogalla/run_storage/Pb-tuning-202302/Pb-Labrador-Sea-{year}_20230217/'
        
    files1 = np.append(files1, np.sort(glob.glob(f'{folder1}/ANHA12_EXH006_5d_*')))
    files2 = np.append(files2, np.sort(glob.glob(f'{folder2}/ANHA12_EXH006_5d_*')))

In [6]:
dates1 = [file.split('ptrc_T_')[1][0:-3] for file in files1]
dates2 = [file.split('ptrc_T_')[1][0:-3] for file in files2]

#### Functions

In [7]:
def plot_layer(file1, file2, n, layer=0, vmin=0, vmax=30):
    
    fig, ax, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/LS.pickle','rb'))
    x, y = proj1(mesh_lon[imin:imax,jmin:jmax], mesh_lat[imin:imax,jmin:jmax]) 
    
    with xr.open_dataset(f'{file1}') as f:
        f1 = f['dissolpb'].values[0,layer,:,:]
    with xr.open_dataset(f'{file2}') as f:
        f2 = f['dissolpb'].values[0,layer,:,:]
    
    CB = proj1.pcolormesh(x, y, (f2-f1)*1e12, cmap=cmocean.cm.thermal, vmin=vmin, vmax=vmax, ax=ax, rasterized=True) 
   
    proj1.contourf(x, y, bathy_masked[layer,imin:imax,jmin:jmax], colors=[land_color], zorder=2, ax=ax) # Mask land at particular depth

    ax.set_title(f'{dates1[n]}', fontsize=6)
    
    # Color bar: --------------------------------------------------------------------------------
    cbaxes = fig.add_axes([0.10, 0.15, 0.02, 0.38]);
    CBar   = plt.colorbar(CB, cax=cbaxes); 
    CBar.set_label('Dissolved Pb [pM]', fontsize=7)
    CBar.ax.tick_params(axis='y', length=0, labelsize=7)
    return fig

#### Figure:

In [ ]:
depth = 10
fig_store = {}
for n, date in enumerate(dates1):
    fig_store[f'{date}'] = plot_layer(files1[n], files2[n], n, layer=depth, vmin=0, vmax=1)    

In [9]:
pdf = matplotlib.backends.backend_pdf.PdfPages("/ocean/brogalla/GEOTRACES/figures/Pb-Baffin-Bay-Atlantic.pdf")
for date in dates1:
    pdf.savefig(fig_store[f'{date}'])
pdf.close()